In [1]:
import re
import nltk
import timeit
import string
import pandas as pd
from nrclex import NRCLex
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from scr.mymodule.database import sql_to_pandas
from nltk.sentiment import SentimentIntensityAnalyzer

C:\Users\Bobby\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Bobby\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

ModuleNotFoundError: No module named 'nrclex'

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bobby\AppData\Roaming\nltk_data...


True

In [16]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    text = ' '.join(tokens)

    return text

def get_sentiment_scores(text):
    # Tokenize the text
    # tokens = nltk.word_tokenize(text)

    # Perform part-of-speech tagging
    # pos_tags = nltk.pos_tag(tokens)

    # Perform sentiment analysis
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)

    # Calculate emotion scores
    positive_score = sentiment_scores['pos']
    negative_score = sentiment_scores['neg']
    neutral_score = sentiment_scores['neu']

    return positive_score, negative_score, neutral_score

def process_text(text):
    if isinstance(text, str):
        return NRCLex(text).affect_frequencies
    else:
        return {}

def get_affect_scores(df):
    # affect_columns = ['fear', 'anger', 'anticipation', 'trust', 'surprise', 'positive', 'negative', 'sadness', 'disgust', 'joy']
    
    texts = df['processed_text'].tolist()

    with Pool() as pool:
        affect_scores = pool.map(process_text, texts)

    df['affect_columns'] = affect_scores #pd.DataFrame(affect_scores, index=df.index)
    
    return df

In [6]:
query = \
"""
SELECT author, "month", author_month_text
FROM public.user_monthly_text

"""

In [7]:
df = sql_to_pandas(query)
df.head(2)

,author,month,author_month_text
0,11Two3,2022-08-01 00:00:00+02:00,do you want a relationship with an equal that ...
1,11Veritas,2021-08-01 00:00:00+02:00,honestly if you became your countrys most deco...


In [8]:
df.shape

(758368, 3)

In [10]:
start_time = timeit.default_timer()

# Code section you want to measure
# Apply your preprocessing function to the author_month_text column
df['processed_text'] = df['author_month_text'].apply(preprocess_text)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 918.748241086 seconds


In [11]:
df.to_csv('data/processed_text.csv', index=False)

In [13]:
start_time = timeit.default_timer()

# Code section you want to measure
df[['positive_score', 'negative_score', 'neutral_score']] = df['processed_text'].apply(get_sentiment_scores).apply(pd.Series)

end_time = timeit.default_timer()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Execution time: 8889.964741812999 seconds


In [14]:
# df.to_csv('data/processed_text_sentiment.csv', index=False)

In [36]:
df.shape

(758368, 7)

In [37]:
(3/1000)*758368/60

37.918400000000005

In [31]:
df_part = df.iloc[0:1000]
print(df_part.shape)
df_part.head(2)

(1000, 7)


,author,month,author_month_text,processed_text,positive_score,negative_score,neutral_score
0,11Two3,2022-08-01 00:00:00+02:00,do you want a relationship with an equal that ...,want relationship equal relate share value int...,0.261,0.097,0.642
1,11Veritas,2021-08-01 00:00:00+02:00,honestly if you became your countrys most deco...,honestly became country decorated olympian did...,0.214,0.000,0.786


In [32]:
df_part = df_part.copy()
df_part.loc[:, 'affect_frequencies'] = df_part['processed_text'].apply(process_text)


In [33]:
df_part.head(2)

,author,month,author_month_text,processed_text,positive_score,negative_score,neutral_score,affect_frequencies
0,11Two3,2022-08-01 00:00:00+02:00,do you want a relationship with an equal that ...,want relationship equal relate share value int...,0.261,0.097,0.642,"{'fear': 0.036585365853658534, 'anger': 0.0731..."
1,11Veritas,2021-08-01 00:00:00+02:00,honestly if you became your countrys most deco...,honestly became country decorated olympian did...,0.214,0.000,0.786,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."


In [35]:
import multiprocessing

def process_text(text):
    if isinstance(text, str):
        return NRCLex(text).affect_frequencies
    else:
        return {}

def process_text_parallel(text):
    pool = multiprocessing.Pool()
    results = pool.map(process_text, text)
    pool.close()
    pool.join()
    return results

df_part['affect_frequencies'] = process_text_parallel(df_part['processed_text'])

Process SpawnPoolWorker-116:
Process SpawnPoolWorker-115:
Process SpawnPoolWorker-117:
Process SpawnPoolWorker-118:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/piyush/opt/anaconda3/lib/python3.9/multiprocessing/qu

KeyboardInterrupt: 

In [18]:
df_final.to_csv('data/processed_text_affect.csv', index=False)

NameError: name 'df_final' is not defined